In [3]:
import numpy as np
import os
import sys
import psutil
import nibabel as nib
from time import time
import json
import matplotlib.pyplot as plt
from contextlib import contextmanager
import warnings
warnings.filterwarnings("ignore")

from shutil import copyfile
import platform
if platform.system() != 'Windows':
    sys.path.insert(0, '/home/users/brezovec/.local/lib/python3.6/site-packages/lib/python/')
    import ants
import brainsss
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import AgglomerativeClustering
import scipy
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import fcluster
from scipy.cluster import hierarchy
import matplotlib as mpl
from matplotlib.pyplot import cm
import random
from scipy.stats import sem
import time
import h5py
import ants
import nibabel as nib
import pandas as pd

import matplotlib
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import interp1d as interp1d

In [4]:
from sklearn.feature_extraction import image
import imageio
from tqdm import tqdm
import argparse
import subprocess
import time
from scipy.ndimage import gaussian_filter1d
import bigbadbrain as bbb
import dataflow as flow
import brainsss
import fcntl
from scipy.ndimage import gaussian_filter1d

In [5]:
import imageio
from tqdm import tqdm

In [6]:
import argparse
import subprocess
import time
from scipy.ndimage import gaussian_filter1d



In [7]:
import bigbadbrain as bbb
import dataflow as flow
import brainsss
import fcntl

In [8]:
from scipy.ndimage import gaussian_filter1d

In [9]:
file_path = "/oak/stanford/groups/trc/data/yandan/2P_Imaging/2023August/aggregated_fly/"

In [10]:
output_directory = '/oak/stanford/groups/trc/data/yandan/2P_Imaging/2023August/aggregated_fly'

In [11]:
# superslice_0.nii
# stack0.npy

In [12]:
dataset_path = '/oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset'

In [13]:
def load_fictrac(directory, file='fictrac.dat'):
    """ Loads fictrac data from .dat file that fictrac outputs.

    To-do: change units based on diameter of ball etc.
    For speed sanity check, instead remove bad frames so we don't have to throw out whole trial.

    Parameters
    ----------
    directory: string of full path to file
    file: string of file name

    Returns
    -------
    fictrac_data: pandas dataframe of all parameters saved by fictrac """

    for item in os.listdir(directory):
      if '.dat' in item:
        file = item

    with open(os.path.join(directory, file),'r') as f:
        df = pd.DataFrame(l.rstrip().split() for l in f)

        # Name columns
        df = df.rename(index=str, columns={0: 'frameCounter',
                                       1: 'dRotCamX',
                                       2: 'dRotCamY',
                                       3: 'dRotCamZ',
                                       4: 'dRotScore',
                                       5: 'dRotLabX',
                                       6: 'dRotLabY',
                                       7: 'dRotLabZ',
                                       8: 'AbsRotCamX',
                                       9: 'AbsRotCamY',
                                       10: 'AbsRotCamZ',
                                       11: 'AbsRotLabX',
                                       12: 'AbsRotLabY',
                                       13: 'AbsRotLabZ',
                                       14: 'positionX',
                                       15: 'positionY',
                                       16: 'heading',
                                       17: 'runningDir',
                                       18: 'speed',
                                       19: 'integratedX',
                                       20: 'integratedY',
                                       21: 'timeStamp',
                                       22: 'sequence'})

        # Remove commas
        for column in df.columns.values[:-1]:
            df[column] = [float(x[:-1]) for x in df[column]]

        fictrac_data = df
                
    # sanity check for extremely high speed (fictrac failure)
    speed = np.asarray(fictrac_data['speed'])
    max_speed = np.max(speed)
    if max_speed > 10:
        raise Exception('Fictrac ball tracking failed (reporting impossibly high speed).')
    return fictrac_data


In [14]:
def load_timestamps(directory, file='functional.xml'):
    """ Parses a Bruker xml file to get the times of each frame, or loads h5py file if it exists.

    First tries to load from 'timestamps.h5' (h5py file). If this file doesn't exist
    it will load and parse the Bruker xml file, and save the h5py file for quick loading in the future.

    Parameters
    ----------
    directory: full directory that contains xml file (str).
    file: Defaults to 'functional.xml'

    Returns
    -------
    timestamps: [t,z] numpy array of times (in ms) of Bruker imaging frames.

    """
    try:
        print('Trying to load timestamp data from hdf5 file.')
        with h5py.File(os.path.join(directory, 'timestamps.h5'), 'r') as hf:
            timestamps = hf['timestamps'][:]

    except:
        print('Failed. Extracting frame timestamps from bruker xml file.')
        xml_file = os.path.join(directory, file)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        timestamps = []
        
        sequences = root.findall('Sequence')
        for sequence in sequences:
            frames = sequence.findall('Frame')
            for frame in frames:
                filename = frame.findall('File')[0].get('filename')
                time = float(frame.get('relativeTime'))
                timestamps.append(time)
        timestamps = np.multiply(timestamps, 1000)

        if len(sequences) > 1:
            timestamps = np.reshape(timestamps, (len(sequences), len(frames)))
        else:
            timestamps = np.reshape(timestamps, (len(frames), len(sequences)))

        ### Save h5py file ###
        with h5py.File(os.path.join(directory, 'timestamps.h5'), 'w') as hf:
            hf.create_dataset("timestamps", data=timestamps)
    
    print('Success.')
    return timestamps

In [15]:
class Printlog():
    '''
    for printing all processes into same log file on sherlock
    '''
    def __init__(self, logfile):
        self.logfile = logfile
    def print_to_log(self, message):
        with open(self.logfile, 'a+') as f:
            fcntl.flock(f, fcntl.LOCK_EX)
            f.write(message)
            f.write('\n')
            fcntl.flock(f, fcntl.LOCK_UN)

In [37]:
#####################
### Main Function ###
#####################

def main(args):


    ### This fly class helps organize data for each fly
    class Fly:
        def __init__ (self, fly_name, fly_idx):
            self.dir = os.path.join(dataset_path, fly_name, 'func_0')
            self.fly_idx = fly_idx
            self.fly_name = fly_name
            self.maps = {}
        def load_timestamps (self):
            self.timestamps = load_timestamps(os.path.join(self.dir, 'imaging'))
        def load_fictrac (self):
            self.fictrac = Fictrac(self, self.dir, self.timestamps)
        def load_brain_slice (self):
            self.brain = brain[self.fly_idx,:,:,:]
        def load_anatomy (self):
            to_load = os.path.join(dataset_path, self.fly_name, 'warp', 'anat-to-meanbrain.nii')
            self.anatomy = np.array(nib.load(to_load).get_data(), copy=True)
        def get_cluster_averages (self, cluster_model_labels, n_clusters):
            neural_data = self.brain.reshape(-1, 3384)
            signals = []
            self.cluster_indicies = []
            for cluster_num in range(n_clusters):
                cluster_indicies = np.where(cluster_model_labels==cluster_num)[0]
                mean_signal = np.mean(neural_data[cluster_indicies,:], axis=0)
                signals.append(mean_signal)
                self.cluster_indicies.append(cluster_indicies) # store for later
            self.cluster_signals=np.asarray(signals)
        def get_cluster_id (self, x, y):
            ax_vec = x*128 + y
            for i in range(n_clusters):
                if ax_vec in self.cluster_indicies[i]:
                    cluster_id = i
                    break
            return cluster_id
        def get_stimulus_start_time_and_ft_frame(self):
            vision_path = os.path.join(self.dir, 'visual')
            ### Load Photodiode ###
            t, ft_triggers, pd1, pd2 = brainsss.load_photodiode(vision_path)
            stimulus_start_times = brainsss.extract_stim_times_from_pd(pd2, t)
            assert len(stimulus_start_times) == 1
            stimulus_start_time = stimulus_start_times[0]
            
            # Find the indices where the fictrac trigger signal rises
            ft_frame_indices = np.where(np.diff(ft_triggers) > 1)[0] + 1
            ft_frame_times = t[ft_frame_indices] / 1000 # ms -> s
            
            # Find fictrac frame index when stimulus starts
            ft_frame_index_when_stim_starts = np.argmin(np.abs(ft_frame_times - stimulus_start_time))
            
            return stimulus_start_time, ft_frame_index_when_stim_starts
                        
    ### This fictrac class helps process behavior data
    class Fictrac:
        def __init__ (self, fly, fly_dir, timestamps):
            self.fly = fly
            self.fictrac_raw = load_fictrac(os.path.join(fly_dir, 'fictrac'))
            self.timestamps = timestamps
        def make_interp_object(self, behavior):
            # Create camera timepoints
            fps=100
            camera_rate = 1/fps * 1000 # camera frame rate in ms
            expt_len = 1000*30*60
            x_original = np.arange(0,expt_len,camera_rate)

            # Smooth raw fictrac data
            fictrac_smoothed = scipy.signal.savgol_filter(np.asarray(self.fictrac_raw[behavior]),25,3)

            # Create interp object with camera timepoints
            fictrac_interp_object = interp1d(x_original, fictrac_smoothed, bounds_error = False)
            return fictrac_interp_object

        def pull_from_interp_object(self, interp_object, timepoints):
            new_interp = interp_object(timepoints)
            np.nan_to_num(new_interp, copy=False);
            return new_interp

        def interp_fictrac(self, z):
            vel_behaviors = ['dRotLabY', 'dRotLabZ']; shorts = ['Y', 'Z']
            self.fictrac = {}

            for behavior, short in zip(vel_behaviors, shorts):
                interp_object = self.make_interp_object(behavior)
                self.fictrac[short + 'i'] = interp_object

                ### Velocity ###
                low_res_behavior = self.pull_from_interp_object(interp_object, self.timestamps[:,z])
                self.fictrac[short] = low_res_behavior#/np.std(low_res_behavior)
                
                ### Clipped Velocities ###
                self.fictrac[short + '_pos'] = np.clip(self.fictrac[short], a_min=0, a_max=None)
                self.fictrac[short + '_neg'] = np.clip(self.fictrac[short], a_min=None, a_max=0)*-1
                
                ### Acceleration ###
                high_res_behavior = self.pull_from_interp_object(interp_object, high_res_timepoints)
                self.fictrac[short + 'h'] = high_res_behavior/np.std(high_res_behavior)

                accel = scipy.signal.savgol_filter(np.diff(high_res_behavior),25,3)
                accel = np.append(accel, 0)
                interp_object = interp1d(high_res_timepoints, accel, bounds_error = False)
                acl = interp_object(self.timestamps[:,z])
                acl[-1] = 0
                self.fictrac[short + 'a'] = acl#/np.std(acl)
                
                ### Clipped Acceleration ###
                self.fictrac[short + 'a' + '_pos'] = np.clip(self.fictrac[short + 'a'], a_min=0, a_max=None)
                self.fictrac[short + 'a' + '_neg'] = np.clip(self.fictrac[short + 'a'], a_min=None, a_max=0)*-1

            self.fictrac['YZ'] = np.sqrt(np.power(self.fictrac['Y'],2), np.power(self.fictrac['Z'],2))
            self.fictrac['YZh'] = np.sqrt(np.power(self.fictrac['Yh'],2), np.power(self.fictrac['Zh'],2))
            
            # Heading, h
            # Create camera timepoints
            fps=100
            camera_rate = 1/fps * 1000 # camera frame rate in ms
            expt_len = 1000*30*60
            x_original = np.arange(0,expt_len,camera_rate)

            # Smooth raw fictrac data
            fictrac_smoothed = scipy.signal.savgol_filter(np.asarray(self.fictrac_raw[behavior]),25,3)

            # Subtract heading at stimulus start
            stimulus_start_time, stimulus_start_ft_frame = self.fly.get_stimulus_start_time_and_ft_frame()
            fictrac_smoothed -= fictrac_smoothed[stimulus_start_ft_frame]
            
            # Create interp object with camera timepoints
            fictrac_interp_object = interp1d(x_original, fictrac_smoothed, bounds_error = False)

            interp_object = self.make_interp_object('heading')
            self.fictrac['h' + 'i'] = interp_object
            low_res_behavior = self.pull_from_interp_object(interp_object, self.timestamps[:,z])
            self.fictrac['h'] = low_res_behavior#/np.std(low_res_behavior)

    ###############################################
    ### Parse the dictionary of input arguments ###
    ###############################################

    logfile = args['logfile']
    save_directory = args['save_directory']
    z = args['z']

    ### this pringlogging is custom to our system
    printlog = getattr(Printlog(logfile=logfile), 'print_to_log')
    printlog('{}'.format(z))

    fly_names = ['fly_287', 'fly_308', 'fly_309', 'fly_311']
    dataset_path = "/oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset"
    expt_len = 1000*30*60
    resolution = 10
    high_res_timepoints = np.arange(0,expt_len,resolution) #0 to last time at subsample res

    #######################
    ### Load Superslice ###
    #######################
    ### A superslice is a single z-plane but all flies have already been concatenated along an axis of this array
    brain_file = "/oak/stanford/groups/trc/data/yandan/2P_Imaging/2023August/aggregated_fly/stack{}.npy".format(z)
    brain = np.array(np.load(brain_file), copy=True)
    # Delete a fly that is in the superslice but was excluded from all analysis due to not passing quality control
    # fly_idx_delete = 3 #(fly_095)
    # brain = np.delete(brain, fly_idx_delete, axis=-1) #### DELETING FLY_095 ####

    #####################
    ### Load Clusters ###
    #####################

    # these are the supervoxels
    n_clusters = 2000
    labels_file = '/oak/stanford/groups/trc/data/yandan/2P_Imaging/2023August/aggregated_fly/cluster_labels.npy'
    cluster_model_labels = np.load(labels_file)
    cluster_model_labels = cluster_model_labels[z,:]

    ###################
    ### Build Flies ###
    ###################
    ### loop  over flies and load and process neural and behavior data based on classes defined above
    flies = {}
    for i, fly in enumerate(fly_names):
        flies[fly] = Fly(fly_name=fly, fly_idx=i)
        flies[fly].load_timestamps()
        flies[fly].load_fictrac()
        flies[fly].fictrac.interp_fictrac(z)
        flies[fly].load_brain_slice()
        flies[fly].get_cluster_averages(cluster_model_labels, n_clusters)

    #####################
    ### Pool behavior ###
    #####################
    ### concatentate behavior across flies
    not_clipped_behaviors = ['Y', 'Z', 'Ya', 'Za', 'h']
    clipped_behaviors = ['Y_pos', 'Y_neg',
                         'Z_pos', 'Z_neg',
                         'Ya_pos', 'Ya_neg',
                         'Za_pos', 'Za_neg']
    all_behaviors = not_clipped_behaviors + clipped_behaviors

    pooled_behavior = {}
    for j, behavior in enumerate(all_behaviors):
        pooled_behavior[behavior] = []
        for i,fly in enumerate(flies):
            pooled_behavior[behavior].append(flies[fly].fictrac.fictrac[behavior])
        pooled_behavior[behavior] = np.asarray(pooled_behavior[behavior]).flatten()

    save_file = os.path.join(save_directory, f'z{z}_pooled_behavior')
    np.save(save_file, pooled_behavior)

    ###################
    ### Correlation ###
    ###################
    
    for behavior_to_corr in all_behaviors:
        r_values = []
        p_values = []
        # looping over supervoxels in a given slice
        for cluster in range(n_clusters):
            pooled_activity = []
            for fly in flies:
                pooled_activity.append(flies[fly].cluster_signals[cluster])
            pooled_activity = np.asarray(pooled_activity).flatten()

            Y = pooled_activity
            X = pooled_behavior[behavior_to_corr]

            r, p = scipy.stats.pearsonr(X, Y) # calculate correlation
            r_values.append(r)
            p_values.append(p)

            # CONFIRMED IDENTICAL ON 20210101
            # Calculate p-value a 2nd way for comparison
            # n = len(Y)
            # t = (r*np.sqrt(n-2))/(np.sqrt(1-r**2))
            # p_manual = scipy.stats.t.sf(abs(t), df=n-2)*2
            # p_values_t_test.append(p_manual)

        #####################
        ### Save Map Data ###
        #####################
        save_file = os.path.join(save_directory, 'z{}_rvalues_{}'.format(behavior_to_corr, z))
        np.save(save_file, np.asarray(r_values))

        save_file = os.path.join(save_directory, 'z{}_pvalues_{}'.format(behavior_to_corr, z))
        np.save(save_file, np.asarray(p_values))

In [38]:
for z in tqdm(range(49)):
    print(f'===== z: {z} =====')
    args = {'z': z, 
            'logfile': "/oak/stanford/groups/trc/data/yandan/2P_Imaging/2023August/aggregated_fly/log.log",
            'save_directory': "/oak/stanford/groups/trc/data/yandan/2P_Imaging/2023August/aggregated_fly"}

    main(args)

  0%|          | 0/49 [00:00<?, ?it/s]

===== z: 0 =====
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done


  2%|▏         | 1/49 [00:30<24:07, 30.15s/it]

===== z: 1 =====
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done


  4%|▍         | 2/49 [00:58<23:00, 29.37s/it]

...

 96%|█████████▌| 47/49 [27:38<00:57, 28.70s/it]

===== z: 47 =====
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done


 98%|█████████▊| 48/49 [28:11<00:30, 30.17s/it]

===== z: 48 =====
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done
Trying to load timestamp data from hdf5 file.
Success.
loading photodiode data... done


100%|██████████| 49/49 [28:38<00:00, 35.07s/it]


In [7]:
def create_clusters(brain, n_clusters):
    t0 = time.time()
    clustering_dir = "/oak/stanford/groups/trc/data/yandan/2P_Imaging/2023August/aggregated_fly"
    super_to_cluster = brain.reshape(-1, 3384*4)
    connectivity = image.grid_to_graph(256,128) ### this is based on image dimensions of a single slice
    cluster_model = AgglomerativeClustering(n_clusters=n_clusters,
                                    memory=clustering_dir,
                                    linkage='ward',
                                    connectivity=connectivity)
    cluster_model.fit(super_to_cluster)
    print('Duration: {}'.format(time.time()-t0))
    return cluster_model

labels = []
### loop over z-slices
for z in range(49):
    print(z)
    t0 = time.time()
    brain_file = "/oak/stanford/groups/trc/data/yandan/2P_Imaging/2023August/aggregated_fly/stack{}.npy".format(z)
    brain = np.array(np.load(brain_file), copy=True)
    # Delete a fly that is in the superslice but was excluded from all analysis due to not passing quality control
    print(f'Duration: {time.time()-t0}')
    # brain = np.delete(brain, fly_idx_delete, axis=-1) #### DELETING FLY_095 ####

    n_clusters = 2000 # you have to tell the function how many supervoxels you want. I swept this number.
    cluster_model = create_clusters(brain, n_clusters)
    labels.append(cluster_model.labels_)
    
save_file = '/oak/stanford/groups/trc/data/yandan/2P_Imaging/2023August/aggregated_fly/final_9_cluster_labels_2000'
np.save(save_file, np.asarray(labels))

0


KeyboardInterrupt: 